# RAG Application with AWS Bedrock & ChromaDB (Cloud)
## Phase 1: Setup & Configuration
This notebook covers the setup of dependencies, configuration of credentials, and initialization of AWS Bedrock and ChromaDB Cloud clients.

In [ ]:
# Step 1: Install Dependencies
# Using %pip ensures packages are installed in the current Jupyter kernel
%pip install boto3 chromadb langchain langchain-community langchain-aws langchain-text-splitters dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\hayde\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
# Step 2: Configuration & Variables
import os
import chromadb
from chromadb.config import Settings
from langchain_experimental.text_splitter import SemanticChunker

# --- AWS Configuration ---
# PLEASE REPLACE WITH YOUR ACTUAL CREDENTIALS
AWS_ACCESS_KEY_ID = os.environ.get["AWS_ACCESS_KEY_ID", "AKIAXXXEXAMPLE"]
AWS_SECRET_ACCESS_KEY = os.environ.get["AWS_SECRET_ACCESS_KEY", "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"]
AWS_REGION = "us-west-2"

# --- Bedrock Model Configuration ---
# Using a stable Claude 3 Sonnet ID which is widely available in us-west-2
BEDROCK_MODEL_ID = "anthropic.claude-3-sonnet-20240229-v1:0"

# --- ChromaDB Cloud Configuration ---
# Sign up at https://trychroma.com to get your API Token
CHROMA_DATABASE = "sentio" # Usually 'default_database'
CHROMA_COLLECTION_NAME = "rag_collection"

# Apply Environment Variables for Boto3
os.environ["AWS_ACCESS_KEY_ID"] = AWS_ACCESS_KEY_ID
os.environ["AWS_SECRET_ACCESS_KEY"] = AWS_SECRET_ACCESS_KEY
os.environ["AWS_DEFAULT_REGION"] = AWS_REGION

print("Configuration Loaded.")

Configuration Loaded.


In [ ]:
# Step 3: Initialize Clients
import boto3
import chromadb

print("1. Initializing Boto3 Session...")
try:
    session = boto3.Session(
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        region_name=AWS_REGION
    )
    bedrock_client = session.client("bedrock-runtime")
    print("   ✅ Bedrock Client Initialized successfully.")
except Exception as e:
    print(f"   ❌ Error initializing Bedrock: {e}")

print("\n2. Initializing ChromaDB Cloud Client...")
try:
    # Initialize CloudClient specifically for Chroma Cloud
    chroma_client = chromadb.PersistentClient(path="./my_chroma_data")
    
    # Get or create the collection
    collection = chroma_client.get_or_create_collection(name=CHROMA_COLLECTION_NAME)
    print(f"   ✅ Connected to Chroma Cloud. Collection '{CHROMA_COLLECTION_NAME}' ready.")
    print(f"   ℹ️ Current Collection Count: {collection.count()}")
except Exception as e:
    print(f"   ❌ Error initializing ChromaDB Cloud: {e}")

1. Initializing Boto3 Session...
   ✅ Bedrock Client Initialized successfully.

2. Initializing ChromaDB Cloud Client...
   ✅ Connected to Chroma Cloud. Collection 'rag_collection' ready.
   ℹ️ Current Collection Count: 0


## Phase 2: Data Ingestion & Chunking
We will read text files from the `files/` directory, chunk them using LangChain's `RecursiveCharacterTextSplitter`, save the chunks to `files/chunked/`, and verify the output.

In [5]:
# Step 4: Setup Directories
import os

SOURCE_DIR = "files"
CHUNKED_DIR = os.path.join(SOURCE_DIR, "chunked")

# Create chunked directory if it doesn't exist
if not os.path.exists(CHUNKED_DIR):
    os.makedirs(CHUNKED_DIR)
    print(f"✅ Created directory: {CHUNKED_DIR}")
else:
    print(f"ℹ️ Directory exists: {CHUNKED_DIR}")

# List source files (excluding directory or hidden files)
source_files = [f for f in os.listdir(SOURCE_DIR) if os.path.isfile(os.path.join(SOURCE_DIR, f)) and not f.startswith('.')]
print(f"Found {len(source_files)} files in {SOURCE_DIR}: {source_files[:5]} ...")

ℹ️ Directory exists: files\chunked
Found 95 files in files: ['academic_and_professional_preparation_requirements_for_faculty.txt', 'academic_credit_policy.txt', 'academic_integrity_monitoring.txt', 'academic_progress_policy_as_bus_jepson.txt', 'acceptable_use_policy.txt'] ...


In [6]:
import nltk
nltk.download("punkt")
nltk.download("punkt_tab")

#from torch.optim.lr_scheduler import LRScheduler

from nltk.tokenize import sent_tokenize
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hayde\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\hayde\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
C:\Users\hayde\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\hayde\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [7]:
class SemanticChunker:
    def __init__(
        self,
        model_name="all-MiniLM-L6-v2",
        similarity_threshold=0.75,
        max_chunk_sentences=10
    ):
        self.model = SentenceTransformer(model_name)
        self.similarity_threshold = similarity_threshold
        self.max_chunk_sentences = max_chunk_sentences

    def chunk(self, text):
        sentences = sent_tokenize(text)

        if len(sentences) == 0:
            return []

        embeddings = self.model.encode(sentences)

        chunks = []
        current_chunk = [sentences[0]]
        current_embeddings = [embeddings[0]]

        for i in range(1, len(sentences)):
            similarity = cosine_similarity(
                [embeddings[i]],
                [np.mean(current_embeddings, axis=0)]
            )[0][0]

            # Start new chunk if topic shift detected
            if (
                similarity < self.similarity_threshold
                or len(current_chunk) >= self.max_chunk_sentences
            ):
                chunks.append(" ".join(current_chunk))
                current_chunk = [sentences[i]]
                current_embeddings = [embeddings[i]]
            else:
                current_chunk.append(sentences[i])
                current_embeddings.append(embeddings[i])

        # Final chunk
        if current_chunk:
            chunks.append(" ".join(current_chunk))

        return chunks


In [8]:
# Step 5: Load, Chunk, and Save Files
from chromadb.utils import embedding_functions
try:
    # Try modern import first
    from langchain_text_splitters import RecursiveCharacterTextSplitter
except ImportError:
    # Fallback to legacy import
    try:
        from langchain.text_splitter import RecursiveCharacterTextSplitter
    except ImportError:
        # Last resort
        from langchain.text_splitter import RecursiveCharacterTextSplitter

embedding_function = embedding_functions.DefaultEmbeddingFunction()

chunker = SemanticChunker(similarity_threshold=0.7)

# Initialize Splitter (prioritize sentence boundaries)
size_splitter = RecursiveCharacterTextSplitter(
    separators=[". ", "? ", "! ", "\n", " ", ""],
    chunk_size=1000,      # Characters (~200 tokens)
    chunk_overlap=100,    # Overlap to maintain context
    length_function=len,
    is_separator_regex=False
)

total_chunks_processed = 0

print("Starting chunking process...\n")

for file_name in source_files:
    file_path = os.path.join(SOURCE_DIR, file_name)
    
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
            
        # Create Chunks
        #semantic_chunks = chunker.create_documents([text])
        
        semantic_chunks = chunker.chunk(text)
        chunks = size_splitter.split_text("\n".join(semantic_chunks))
        
        # Save each chunk with metadata in filename
        # Format: ch{index}-{original_name}-{metadata}.txt
        base_name = os.path.splitext(file_name)[0]
        
        for i, chunk_content in enumerate(chunks):
            # Metadata example: length of chunk
            metadata_str = f"len{len(chunk_content)}"
            chunk_filename = f"ch{i+1}-{base_name}-{metadata_str}.txt"
            chunk_path = os.path.join(CHUNKED_DIR, chunk_filename)
            
            with open(chunk_path, 'w', encoding='utf-8') as chunk_file:
                chunk_file.write(chunk_content)
                
        print(f"✅ {file_name}: Created {len(chunks)} chunks.")
        total_chunks_processed += len(chunks)
        
    except Exception as e:
        print(f"❌ Error processing {file_name}: {e}")

print(f"\n🎉 Total Chunks Created: {total_chunks_processed}")

Starting chunking process...

✅ academic_and_professional_preparation_requirements_for_faculty.txt: Created 8 chunks.
✅ academic_credit_policy.txt: Created 52 chunks.
✅ academic_integrity_monitoring.txt: Created 7 chunks.
✅ academic_progress_policy_as_bus_jepson.txt: Created 12 chunks.
✅ acceptable_use_policy.txt: Created 21 chunks.
✅ access_to_electronic_files_policy.txt: Created 14 chunks.
✅ additional_compensation_for_staff.txt: Created 13 chunks.
✅ administrative_data_management_policy.txt: Created 28 chunks.
✅ alcohol_and_drug_policy.txt: Created 59 chunks.
✅ appointment_of_endowed_chairs.txt: Created 16 chunks.
✅ benefits_for_10-month_staff_and_faculty_on_9-month_contracts.txt: Created 4 chunks.
✅ bereavement_leave_policy.txt: Created 4 chunks.
✅ board_of_trustees_conflict_of_interest_policy.txt: Created 36 chunks.
✅ board_of_trustees_contract_approval_and_signature_authority.txt: Created 21 chunks.
✅ business_meals_policy.txt: Created 19 chunks.
✅ campus_ministry_policy.txt: Cre

In [10]:
# Step 6: Verify a Sample Chunk
# Check one of the generated files to ensure content is correct
if os.listdir(CHUNKED_DIR):
    sample_chunk = os.listdir(CHUNKED_DIR)[0]
    sample_path = os.path.join(CHUNKED_DIR, sample_chunk)
    
    print(f"--- Content of {sample_chunk} ---")
    with open(sample_path, 'r', encoding='utf-8') as f:
        print(f.read()[:500]) # Print first 500 chars
    print("\n--- End of Sample ---")
else:
    print("No chunks found to verify.")

--- Content of ch1-academic_and_professional_preparation_requirements_for_faculty-len495.txt ---
university of richmond | 1
university of richmond
policy manual
purpose:
this policy is designed to ensure that faculty at the university of richmond have the highest quality
preparation to accomplish the educational mission of the university and to ensure that the university of
richmond's academic and professional preparation requirements for full-time and part-time faculty conform
to the criteria established by the commission on colleges of the southern association of colleges and
schools

--- End of Sample ---


## Phase 3: Embeddings & Vector Store
We will now read the chunked files we just created, generate embeddings (handled automatically by Chroma's default embedding function), and upsert them into the ChromaDB Cloud collection.

> **Note:** We are using ChromaDB's default embedding model (`all-MiniLM-L6-v2`) which is built into the client. No extra API calls to Bedrock are needed for *embedding* in this setup, saving costs.

In [11]:
# Step 7: Prepare Data for Embedding
import uuid
import re

chunked_files = [f for f in os.listdir(CHUNKED_DIR) if f.endswith('.txt')]

documents = []
metadatas = []
ids = []

print(f"Found {len(chunked_files)} chunk files to process.")

for file_name in chunked_files:
    file_path = os.path.join(CHUNKED_DIR, file_name)
    
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
            
        # Parse Metadata from Filename
        # Format: ch{index}-{original_name}-{len}.txt
        # Example: ch1-academic_policy-len495.txt
        try:
            name_no_ext = os.path.splitext(file_name)[0]
            parts = name_no_ext.split('-')
            
            # 1. Chunk Part (first item, e.g., 'ch1')
            chunk_part = int(parts[0].replace('ch', ''))
            
            # 2. Size (last item, e.g., 'len495')
            size = int(parts[-1].replace('len', ''))
            
            # 3. File Name (everything in between)
            original_filename = "-".join(parts[1:-1])
            
            meta = {
                "source": file_name,
                "file_name": original_filename,
                "chunk_part": chunk_part,
                "size": size
            }
        except Exception as e:
            # Fallback if naming convention doesn't match
            print(f"⚠️ Metadata parse warning for {file_name}: {e}")
            meta = {"source": file_name}

        # Add to lists
        documents.append(content)
        metadatas.append(meta)
        ids.append(str(uuid.uuid4()))
        
    except Exception as e:
        print(f"Warning: Could not read {file_name}: {e}")

print(f"Prepared {len(documents)} documents for embedding.")

Found 3810 chunk files to process.
Prepared 3810 documents for embedding.


In [12]:
# Step 8: Add to ChromaDB (Embed & Upsert)
# Batch size limit for Chroma is usually 1000 (we hit 1914!), so we must batch.
print("Upserting documents to ChromaDB Collection in batches...")

BATCH_SIZE = 100  # Safe batch size
total_docs = len(documents)

try:
    for i in range(0, total_docs, BATCH_SIZE):
        batch_docs = documents[i : i + BATCH_SIZE]
        batch_metas = metadatas[i : i + BATCH_SIZE]
        batch_ids = ids[i : i + BATCH_SIZE]
        
        collection.add(
            documents=batch_docs,
            metadatas=batch_metas,
            ids=batch_ids
        )
        print(f"   ✅ Processed batch {i} to {min(i+BATCH_SIZE, total_docs)}")
        
    print(f"\n🎉 Successfully added all {total_docs} documents to ChromaDB!")
    print(f"Final Collection Count: {collection.count()}")
    
except Exception as e:
    print(f"❌ Error adding to ChromaDB: {e}")

Upserting documents to ChromaDB Collection in batches...
   ✅ Processed batch 0 to 100
   ✅ Processed batch 100 to 200
   ✅ Processed batch 200 to 300
   ✅ Processed batch 300 to 400
   ✅ Processed batch 400 to 500
   ✅ Processed batch 500 to 600
   ✅ Processed batch 600 to 700
   ✅ Processed batch 700 to 800
   ✅ Processed batch 800 to 900
   ✅ Processed batch 900 to 1000
   ✅ Processed batch 1000 to 1100
   ✅ Processed batch 1100 to 1200
   ✅ Processed batch 1200 to 1300
   ✅ Processed batch 1300 to 1400
   ✅ Processed batch 1400 to 1500
   ✅ Processed batch 1500 to 1600
   ✅ Processed batch 1600 to 1700
   ✅ Processed batch 1700 to 1800
   ✅ Processed batch 1800 to 1900
   ✅ Processed batch 1900 to 2000
   ✅ Processed batch 2000 to 2100
   ✅ Processed batch 2100 to 2200
   ✅ Processed batch 2200 to 2300
   ✅ Processed batch 2300 to 2400
   ✅ Processed batch 2400 to 2500
   ✅ Processed batch 2500 to 2600
   ✅ Processed batch 2600 to 2700
   ✅ Processed batch 2700 to 2800
   ✅ Process

In [13]:
# Step 9: Verify Embedding with a Test Query
# We will perform a simple similarity search (no LLM yet) to see if we get relevant chunks.

query_text = "What is the main topic of these documents?"  # Replace with a relevant question for your data

print(f"Querying ChromaDB for: '{query_text}'...\n")

results = collection.query(
    query_texts=[query_text],
    n_results=3 # Get top 3 matches
)

if results['documents']:
    for i, doc in enumerate(results['documents'][0]):
        meta = results['metadatas'][0][i]
        print(f"[Result {i+1}]")
        print(f"   File: {meta.get('file_name', 'Unknown')}")
        print(f"   Part: {meta.get('chunk_part', '?')}")
        print(f"   Size: {meta.get('size', '?')}")
        print(f"   Snippet: {doc[:100]}...\n")
else:
    print("No results found. Check if documents were added correctly.")

Querying ChromaDB for: 'What is the main topic of these documents?'...

[Result 1]
   File: hipaa_policy
   Part: 23
   Size: 931
   Snippet: . the library is located in
box for internal access. outdated and superseded materials from the secu...

[Result 2]
   File: hipaa_policy
   Part: 21
   Size: 997
   Snippet: . the university will maintain documentation, in written or electronic form, of policies, procedures...

[Result 3]
   File: policy_on_prohibiting_and_responding_to_sex_discrimination_faculty_staff
   Part: 56
   Size: 168
   Snippet: . review investigative report.
the title ix coordinator shall review the final investigative report
...



## Phase 4: Retrieval & Generation
We implement the custom retrieval logical (with distince threshold filtering) and connect it to AWS Bedrock for the final answer generation.

In [47]:
# Step 10: Custom Retrieval Function
def retrieve_documents(query, n_results=5, threshold=1.5, filter_by=None):
    """
    Retrieve relevant documents with distance threshold filtering.
    
    Args:
        query: The search query string
        n_results: Max results to return initially
        threshold: Max distance (lower = more strict match). 
                   For Cosine distance: 0 is identical, 1 is orthogonal, 2 is opposite.
                   Typical good matches are < 1.0 depending on embedding model.
        filter_by: Metadata filter dict (optional)
    
    Returns:
        List of dicts: {text, source, distance}
    """
    #print(f"Filter by: {filter_by}")
    #Add LLM here to ask what documents have the info
    metadata = collection.get(include=["metadatas"])
    file_names = sorted({
        meta["file_name"]
        for meta in metadata["metadatas"]
        if "file_name" in meta
    })
    # Query Chroma
    results = collection.query(
        query_texts=[query],
        n_results=n_results,
        where=filter_by,
        include=["documents", "metadatas", "distances"]
    )
    #print(results)
    
    docs = []
    
    # Check if we got results
    if results['documents'] and results['documents'][0]:
        # Iterate through the first query's results
        for text, meta, dist in zip(
            results['documents'][0],
            results['metadatas'][0],
            results['distances'][0]
        ):
            # Filter by threshold
            if dist <= threshold:
                docs.append({
                    "text": text,
                    "source": meta.get("source", "unknown"),
                    "distance": dist
                })
                
    print(f"✅ Retrieved {len(docs)} documents (Threshold: {threshold})")
    return docs

In [48]:
# Step 11: RAG Generation Function (Bedrock)
from langchain_aws import ChatBedrock

# Fix for newer LangChain versions (v0.1+)
try:
    from langchain_core.prompts import PromptTemplate
    from langchain_core.runnables import RunnablePassthrough
    from langchain_core.output_parsers import StrOutputParser
except ImportError:
    # Fallback for older versions
    from langchain.prompts import PromptTemplate
    from langchain.schema.runnable import RunnablePassthrough
    from langchain.schema.output_parser import StrOutputParser

# Initialize LLM
llm = ChatBedrock(
    model_id=BEDROCK_MODEL_ID,
    client=bedrock_client,
    model_kwargs={"max_tokens": 1000, "temperature": 0.1} # Claude 3 uses 'max_tokens'
)

def generate_answer(query):
    # 1. Retrieve Context
    results = collection.get(include=["metadatas"])

    unique_filenames = set()

    metadata = results["metadatas"]
    for data in metadata:
        unique_filenames.add(data['file_name'])
    
    #print(unique_filenames)

    source_selection_prompt = f"""
You are given a list of document sources.
Return ONLY the names of the sources that are relevant to the question.
If none are relevant, return an empty list.
Do not explain your reasoning.

Sources:
{', '.join(unique_filenames)}

Question:
{query}

Return format:
source1, source2, source3
    """
    
    source_response = llm.invoke(source_selection_prompt).content.strip()

    if source_response:
        selected_sources = [s.strip() for s in source_response.split(",")]
    else:
        selected_sources = []

    #print(selected_sources)

    relevant_docs = retrieve_documents(query, n_results=5, threshold=1.2, filter_by={"file_name": {"$in": selected_sources}})
    #relevant_docs = retrieve_documents(query, n_results=5, threshold=1.2)
    
    if not relevant_docs:
        return "I could not find any relevant information to answer your question."
    
    # 2. Format Context
    context_text = "\n\n".join([f"[Source: {d['source']}]\n{d['text']}" for d in relevant_docs])
    
    # 3. Construct Prompt
    prompt_template = """
    Human: You are a concise and direct assistant. Use the following pieces of context to answer the question at the end.
    
    Rules for answering:
    1. Be extremely concise.
    2. Do NOT use bullet points or numbered lists. 
    3. Provide a single, direct paragraph.
    4. If you don't know the answer, just say that you don't know.

    Context:
    {context}

    Question: {question}

    Assistant:"""
    
    prompt = PromptTemplate(
        template=prompt_template, 
        input_variables=["context", "question"]
    )
    
    # 4. Invoke Chain manually (since we have custom retrieval logic)
    # We format the prompt first, then send to LLM
    final_prompt = prompt.format(context=context_text, question=query)
    response = llm.invoke(final_prompt)
    
    return response.content

In [49]:
# Step 12: Final Test
query = "What is the policy regarding drug usage?"

print(f"❓ Question: {query}\n")

answer = generate_answer(query)

print("💡 Answer:")
print(answer)

❓ Question: What is the policy regarding drug usage?

✅ Retrieved 5 documents (Threshold: 1.2)
💡 Answer:
The university has a strict policy prohibiting the unlawful manufacture, possession, use, or distribution of illegal drugs, marijuana, and/or alcohol by employees and students on university property or as part of university activities. Violations can result in disciplinary actions ranging from substance education to permanent separation or dismissal from employment. The university reserves the right to conduct drug/alcohol testing for reasonable suspicion and may refer violators for criminal prosecution.
